**AQI Prediction Using Xgboost Regressor**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
from pathlib import Path
file_path = Path('D:\Internship\IBM Internship\Project1\Air-Quality-Index-prediction_with_deployment\ML_Applied_Word\PreProcess\df_cleaned_data.pkl')
with open(file_path,'rb') as file:
    df= pickle.load(file)

<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\ruchi\AppData\Local\Temp\ipykernel_60840\2755762934.py:2: SyntaxWarning: invalid escape sequence '\I'
  file_path = Path('D:\Internship\IBM Internship\Project1\Air-Quality-Index-prediction_with_deployment\ML_Applied_Word\PreProcess\df_cleaned_data.pkl')


In [ ]:
# with open('/content/drive/MyDrive/Google_colab_project/df.pkl','rb') as file:
#     df= pickle.load(file)

In [3]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

### **XGBoost Regressor Implementation without Hyper Tuning**

In [5]:
import xgboost as xgb
xgb_regressor=xgb.XGBRegressor()
xgb_regressor.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [6]:

print("Coefficient of determination R^2 <-- on train set: {}".format(xgb_regressor.score(X_train, y_train)))
print("Coefficient of determination R^2 <-- on train set: {}".format(xgb_regressor.score(X_test, y_test)))

Coefficient of determination R^2 <-- on train set: 0.9986631893048898
Coefficient of determination R^2 <-- on train set: 0.5205096115629093


#### **Model Evaluation**



In [7]:
xgb_prediction=xgb_regressor.predict(X_test)

In [8]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, xgb_prediction))
print('MSE:', metrics.mean_squared_error(y_test, xgb_prediction))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, xgb_prediction)))

MAE: 40.895398597542304
MSE: 3329.3743450855923
RMSE: 57.70073088866026


XGBoost Regressor model has  RMSE(Root Mean Square Error) is around 45 without any hyperparameter tuning. So, Next, we will do hyperparameter tuning to getting a lower RMSE value.

### **XGBoost Regressor Implementation with Hyperparameter Tuning- Randomized SerchCV**


#### Randomized SerchCV-1

In [9]:
xgb_regressor_rcv1=xgb.XGBRegressor()

In [10]:
#Randomized Search CV

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Various learning rate parameters
learning_rate = ['0.05','0.1', '0.2','0.3','0.5','0.6']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
#Subssample parameter values
subsample=[0.7,0.6,0.8]
# Minimum child weight parameters
min_child_weight=[3,4,5,6,7]

In [11]:

# Create the random grid
random_grid_1 = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_depth': max_depth,
               'subsample': subsample,
               'min_child_weight': min_child_weight}

print(random_grid_1)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200], 'learning_rate': ['0.05', '0.1', '0.2', '0.3', '0.5', '0.6'], 'max_depth': [5, 10, 15, 20, 25, 30], 'subsample': [0.7, 0.6, 0.8], 'min_child_weight': [3, 4, 5, 6, 7]}


In [12]:
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations

xgb_rcv1 = RandomizedSearchCV(estimator = xgb_regressor_rcv1, param_distributions = random_grid_1,
                               scoring='neg_mean_squared_error', n_iter = 100, cv = 5, verbose=2, 
                               random_state=42, n_jobs = 1)


In [13]:
xgb_rcv1.fit(X_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END learning_rate=0.3, max_depth=10, min_child_weight=6, n_estimators=1100, subsample=0.7; total time=   0.8s
[CV] END learning_rate=0.3, max_depth=10, min_child_weight=6, n_estimators=1100, subsample=0.7; total time=   0.7s
[CV] END learning_rate=0.3, max_depth=10, min_child_weight=6, n_estimators=1100, subsample=0.7; total time=   1.2s
[CV] END learning_rate=0.3, max_depth=10, min_child_weight=6, n_estimators=1100, subsample=0.7; total time=   0.9s
[CV] END learning_rate=0.3, max_depth=10, min_child_weight=6, n_estimators=1100, subsample=0.7; total time=   1.0s
[CV] END learning_rate=0.3, max_depth=20, min_child_weight=6, n_estimators=600, subsample=0.7; total time=   0.9s
[CV] END learning_rate=0.3, max_depth=20, min_child_weight=6, n_estimators=600, subsample=0.7; total time=   1.6s
[CV] END learning_rate=0.3, max_depth=20, min_child_weight=6, n_estimators=600, subsample=0.7; total time=   1.0s
[CV] END learning_ra

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=100, n_jobs=1,
                   param_distributions={'learning_rate': ['0.05', '0.1', '0.2',
                                                          '0.3', '0.5', '0.6'],
                                        'max_depth': [5, 10, 15, 20, 25, 30],
                                        'min_child_weight': [3, 4, 5, 6, 7],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000, 1100,
                                                         1200],
                                        'subsample': [0.7, 0.6, 0.8]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [14]:
xgb_rcv1_prediction=xgb_rcv1.predict(X_test)
print(xgb_rcv1.best_params_)
print(xgb_rcv1.best_score_)
print('MAE:', metrics.mean_absolute_error(y_test,xgb_rcv1_prediction))
print('MSE:', metrics.mean_squared_error(y_test, xgb_rcv1_prediction))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, xgb_rcv1_prediction)))

{'subsample': 0.6, 'n_estimators': 100, 'min_child_weight': 6, 'max_depth': 15, 'learning_rate': '0.05'}
-2784.6756546135375
MAE: 37.46968136475356
MSE: 2962.3329104100753
RMSE: 54.42731768524033


#### Randomized SerchCV-2

In [15]:
xgb_regressor_rcv2=xgb.XGBRegressor()

In [16]:
#Randomized Search CV

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 400, stop = 800, num = 10)]
# Various learning rate parameters
learning_rate = ['0.08', '0.1','0.12','0.14','0.16']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 20, num = 5)]
# max_depth.append(None)
#Subssample parameter values
subsample=[0.7,0.6,0.8]
# Minimum child weight parameters
min_child_weight=[6,7,8,9]

In [17]:

# Create the random grid
random_grid2= {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_depth': max_depth,
               'subsample': subsample,
               'min_child_weight': min_child_weight}

print(random_grid2)

{'n_estimators': [400, 444, 488, 533, 577, 622, 666, 711, 755, 800], 'learning_rate': ['0.08', '0.1', '0.12', '0.14', '0.16'], 'max_depth': [5, 8, 12, 16, 20], 'subsample': [0.7, 0.6, 0.8], 'min_child_weight': [6, 7, 8, 9]}


In [18]:
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations


xgb_rcv2 = RandomizedSearchCV(estimator = xgb_regressor_rcv2, param_distributions = random_grid2,
                               scoring='neg_mean_squared_error', n_iter = 100, cv = 5, verbose=2, 
                               random_state=100, n_jobs = 1)


In [19]:
xgb_rcv2.fit(X_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END learning_rate=0.12, max_depth=5, min_child_weight=8, n_estimators=577, subsample=0.6; total time=   0.7s
[CV] END learning_rate=0.12, max_depth=5, min_child_weight=8, n_estimators=577, subsample=0.6; total time=   0.5s
[CV] END learning_rate=0.12, max_depth=5, min_child_weight=8, n_estimators=577, subsample=0.6; total time=   0.3s
[CV] END learning_rate=0.12, max_depth=5, min_child_weight=8, n_estimators=577, subsample=0.6; total time=   0.2s
[CV] END learning_rate=0.12, max_depth=5, min_child_weight=8, n_estimators=577, subsample=0.6; total time=   0.2s
[CV] END learning_rate=0.12, max_depth=8, min_child_weight=8, n_estimators=400, subsample=0.7; total time=   0.3s
[CV] END learning_rate=0.12, max_depth=8, min_child_weight=8, n_estimators=400, subsample=0.7; total time=   0.2s
[CV] END learning_rate=0.12, max_depth=8, min_child_weight=8, n_estimators=400, subsample=0.7; total time=   0.3s
[CV] END learning_rate=0.

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=100, n_jobs=1,
                   param_distributions={'learning_rate': ['0.08', '0.1', '0.12',
                                                          '0.14', '0.16'],
                                        'max_depth': [5, 8, 12, 16, 20],
                                        'min_child_weight': [6, 7, 8, 9],
                                        'n_estimators': [400, 444, 488, 533,
                                                         577, 622, 666, 711,
                                                         755, 800],
                                        'subsample': [0.7, 0.6, 0.8]},
                   random_state=100, scoring='neg_mean_squared_error',
                   verbose=2)

In [20]:

xgb_rcv2_prediction=xgb_rcv2.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, xgb_rcv2_prediction))
print('MSE:', metrics.mean_squared_error(y_test, xgb_rcv2_prediction))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, xgb_rcv2_prediction)))

MAE: 38.978764749521154
MSE: 3191.8381380619076
RMSE: 56.496355086517816


In [21]:
print(xgb_rcv2.best_params_)
print(xgb_rcv2.best_score_)

{'subsample': 0.8, 'n_estimators': 755, 'min_child_weight': 7, 'max_depth': 20, 'learning_rate': '0.08'}
-2982.819556571082


### **XGBoost Regressor Implementation with Hyperparameter Tuning-GridSerchCV**

#### GridSerchCV-1


In [22]:
xgb_regressor_gcv1=xgb.XGBRegressor()

In [23]:
## Hyperparameter optimization using GridSearchCV
from sklearn.model_selection import GridSearchCV
## Hyper Parameter Optimization

params_grid_1={
  "loss": ["ls","lad","huber"],
  "n_estimators":[int(x) for x in np.linspace(start = 400, stop = 800, num = 12)],
 'learning_rate': ['0.08', '0.1','0.12','0.14'],
  'max_depth':[int(x) for x in np.linspace(12, 24, num = 5)],
  'subsample':[0.7,0.6,0.8],
  'min_child_weight':[6,7,8]   
}

In [27]:

xgb_gcv1=GridSearchCV( xgb_regressor_gcv1,param_grid=params_grid_1,scoring='neg_mean_squared_error',n_jobs=-1,cv=10,verbose=10)
xgb_gcv1.fit(X_train,y_train)

Fitting 10 folds for each of 6480 candidates, totalling 64800 fits


In [ ]:
from sklearn import metrics
xgb_gcv_prediction_1=xgb_gcv1.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, xgb_gcv_prediction_1))
print('MSE:', metrics.mean_squared_error(y_test, xgb_gcv_prediction_1))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, xgb_gcv_prediction_1)))

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('xgb_gridcv1.pkl', 'wb')

# dump information to that file
pickle.dump(xgb_gcv1, file)

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('xgb_randomcv1.pkl', 'wb')

# dump information to that file
pickle.dump(xgb_rcv1, file)

In [ ]:
print(xgb_gcv1.best_params_)
print(xgb_gcv1.best_score_)